# Wstęp do Uczenia Maszynowego - Lab 5

Wszystkie dzisiejsze metody można znaleźć dokładnie (i przystępnie!) omówione na StatQuest: https://www.youtube.com/user/joshstarmer

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
np.random.seed = 42


In [4]:
data = pd.read_csv('heart.csv')
data.head()

sbp  tobacco   ldl  adiposity  famhist  typea  obesity  alcohol  age  chd
0  160    12.00  5.73      23.11  Present     49    25.30    97.20   52    1
1  144     0.01  4.41      28.61   Absent     55    28.87     2.06   63    1
2  118     0.08  3.48      32.28  Present     52    29.14     3.81   46    0
3  170     7.50  6.41      38.03  Present     51    31.99    24.26   58    1
4  134    13.60  3.50      27.78  Present     60    25.99    57.34   49    1

In [5]:
y = np.array(data['chd'])
X = data.drop(['chd'],axis=1)

In [6]:
map_dict = {'Present': 1, 'Absent':0}
X['famhist'] = X['famhist'].map(map_dict)
X.head()

sbp  tobacco   ldl  adiposity  famhist  typea  obesity  alcohol  age
0  160    12.00  5.73      23.11        1     49    25.30    97.20   52
1  144     0.01  4.41      28.61        0     55    28.87     2.06   63
2  118     0.08  3.48      32.28        1     52    29.14     3.81   46
3  170     7.50  6.41      38.03        1     51    31.99    24.26   58
4  134    13.60  3.50      27.78        1     60    25.99    57.34   49

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

## Ensemble Methods - Komitety
Często kilka osób, które myślą nad daną sprawą, potrafi dać lepszą odpowiedź niż jedna osoba. Nawet jeśli żadna z osób nie jest ekspertem.  
To podejście zadziała, gdy błędny popełniane przez różne modele są od siebie niezależne (w miarę). 

  
Na potrzeby stosowania różnych metod Ensemble Learningu załadujemy sobie już 3 modele z których będziemy potem korzystać.

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

model1 = DecisionTreeClassifier(random_state=1)
model2 = KNeighborsClassifier()
model3 = LogisticRegression(random_state=1, max_iter=1000)
estimators=[('DecisionTree', model1), ('KNN', model2), ('LR', model3)]

### Max Voting
lub Hard Voting - głosowanie większościowe 

In [12]:
from sklearn.ensemble import VotingClassifier

In [13]:
model_hard = VotingClassifier(estimators=estimators, voting='hard')
model_hard.fit(X_train,y_train)

y_hat = model_hard.predict(X_test)
print('accuracy: ', accuracy_score(y_test, y_hat))

# model też ma inną metodę ewaluacji i jest to też accuracy w przypadku klasyfikacji
print('model.score: ', model_hard.score(X_test,y_test))

accuracy:  0.6881720430107527
model.score:  0.6881720430107527


### Averaging
Soft Voting. Nie patrzymy na liczbę głosów, ale na "pewność". Patrzymy na ile pewny jest klasyfikator, że rekord należy do klasy 1. c1 - 90%, c2 - 49%, c3 - 49% 
* hard voting: klasa 0 (1 głos na tak, 2 głosy na nie)
* soft voting: klasa 1 ( (90 + 49 + 49)/3 $\approx$ 63% > 50%)


In [14]:
model_soft = VotingClassifier(estimators=estimators, voting='soft')
model_soft.fit(X_train, y_train)

y_hat = model_soft.predict(X_test)
accuracy_score(y_test, y_hat)

0.6236559139784946

# Weights
Nie musimy zawsze uważać, że każdy klasyfikator ma tyle samo do powiedzenia.

In [16]:
model_soft = VotingClassifier(estimators=estimators, voting='soft', weights=[0.05, 0.05, 0.90])
model_soft.fit(X_train, y_train)

y_hat = model_soft.predict(X_test)
accuracy_score(y_test, y_hat)

0.7419354838709677

### Stacking
<div>
<img src="https://miro.medium.com/max/700/1*RP0pkQEOSrw9_EjFu4w3gg.png" />
</div>
source: https://miro.medium.com/max/700/1*RP0pkQEOSrw9_EjFu4w3gg.png

Bierzemy kilka różnych modeli (base) i jeden meta-model. Meta-model uczy się przewidzieć wynik na podstawie wyników z base.

Zasada kciuka: ostatni model jest raczej prosty (regresja liniowa/logistyczna).  
Więcej (np. o trenowaniu) tutaj: https://machinelearningmastery.com/stacking-ensemble-machine-learning-with-python/


Czy można stackować kilka takich samych modeli?  
Tak, omówimy to za chwilę.

In [17]:
from sklearn.ensemble import StackingClassifier

In [18]:
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
clf.fit(X_train, y_train).score(X_test, y_test)

0.7741935483870968

### Bagging (Bootstrap Aggregating)
Bootstrap - to technika próbkowania, w której tworzymy podzbiory (próby) obserwacji z oryginalnego datasetu, **ze zwracaniem**. Rozmiar podzbiorów jest taki sam jak rozmiar oryginalnego datasetu.

1. Losujemy N **bootstrapowych** prób ze zbioru treningowego
2. Trenujemy niezależnie N "słabych" klasyfikatorów
3. Składamy wyniki "słabych" modeli 
    - **Klasyfikacja:** reguła większościowa / uśrednione prawdopodobieństwo
    - **Regresja:** Uśrednione wartości

In [19]:
from sklearn.ensemble import BaggingClassifier

In [20]:
clf = BaggingClassifier(base_estimator=model1,
                        n_estimators=10, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.6559139784946236

## Random Forest
Najbardziej popularny algorytm Baggingowy. Wiele małych drzew.

Przypomnijmy
#### Zalety drzew
* interpretowalność
* prostota + wizualizacja
* nie trzeba normalizować danych \:)
* odporne na wartości skrajne
* z ich użyciem można wykrywać ważne cechy

#### Wady drzew
* łatwo o overfitting
* drzewo długo rośnie
* starych drzew się nie przesadza - nie da się dotrenować algorytmu po jakimś czasie, trzeba zasadzić nowe 
* jest duża losowość - na tych samych danych algorytm może dawać bardzo różne wyniki

In [21]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators=1000, # Ilość słabych estymatorów
                                  max_depth=2, # Maksymalna wysokość drzewa w słabym estymatorze
                                  min_samples_split = 2, # Minimalna ilość obserwacji wymagana do podziału węzła
                                  max_features = 3, # Maksymalna ilość zmiennych brana pod uwagę przy podziale węzła
                                  random_state=0,
                                  n_jobs = -1)
model_rf.fit(X_train, y_train)
model_rf.score(X_test,y_test)

0.7204301075268817

### Boosting
Boosting działa podobnie jak Bagging z jedną różnicą. Każda kolejna próba bootstrap jest tworzona w taki sposób,  
że losuje z większym prawdopodobieństwiem obserwacje **źle sklasyfikowane**.  
W skrócie: Boosting uczy się na błędach, które popełnił w poprzednich iteracjach.

#### AdaBoost
Najprostsza metoda boostingowa.  
Większe prawdopodobieństwo wylosowania próbek, na których podczas predykcji został popełniony błąd.

In [22]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(random_state=1)
model.fit(X_train, y_train)
model.score(X_test,y_test)

0.7204301075268817

#### Gradient Boosting
Zaczynamy od pojedynczej predykcji (np. średniej) i liczymy różnicę, tzw. residuum.  
Następnie każdy model próbuje przewidzieć residuum.  
W GB każdy model ma taki sam głos.

In [23]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(random_state=1,
                                  learning_rate=0.01)
model.fit(X_train, y_train)
model.score(X_test,y_test)

0.7204301075268817

#### XGBoost
Zaawansowana implementacja Gradient Boostingu

In [24]:
! pip install xgboost
import xgboost
from xgboost import XGBClassifier # Inna paczka niż sklearn!

/hdd/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [25]:
model=XGBClassifier(random_state=1,
                    learning_rate=0.01, # Szybkość "uczenia" się
                    booster='gbtree', # Jaki model wykorzystujemy (drzewo - gbtree, liniowe - gblinear)
                    nround = 100, # Ilość itereacji boosingowych
                    max_depth=4 # Maksymalna głębokość drzewa 
                    )
model.fit(X_train, y_train)
model.score(X_test,y_test)

[12:41:57] WARNING: ../src/learner.cc:576: 
Parameters: { "nround" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:41:57] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/hdd/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/hdd/anaconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.7526881720430108

In [38]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score

In [39]:
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot
import warnings
warnings.filterwarnings('ignore')

In [40]:
# przygotowanie stacking
def get_stacking():
    
    # define the base models
    level0 = list()
    level0.append(('lr', LogisticRegression()))
    level0.append(('cart', DecisionTreeClassifier(random_state=1)))
    level0.append(('knn', KNeighborsClassifier()))
    level0.append(('rf', RandomForestClassifier(n_estimators=1000, # Ilość słabych estymatorów
                                      max_depth=2, # Maksymalna wysokość drzewa w słabym estymatorze
                                      min_samples_split = 2, # Minimalna ilość obserwacji wymagana do podziału węzła
                                      max_features = 3, # Maksymalna ilość zmiennych brana pod uwagę przy podziale węzła
                                      random_state=0,
                                      n_jobs = -1)))
    level0.append(('aboost', AdaBoostClassifier(random_state=1)))
    level0.append(('gboost', GradientBoostingClassifier(random_state=1,
                                      learning_rate=0.01)))
    level0.append(('xgb', XGBClassifier(random_state=1,
                        learning_rate=0.01, # Szybkość "uczenia" się
                        booster='gbtree', # Jaki model wykorzystujemy (drzewo - gbtree, liniowe - gblinear)
                        max_depth=4 # Maksymalna głębokość drzewa 
                        )))
    level0.append(('svm', SVC()))
    level0.append(('bayes', GaussianNB()))
    
    # definicja meta learner model
    level1 = LogisticRegression()
    
    # definicja stacking ensemble
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    
    return model

In [51]:
# lista modeli
def get_models():
    
    models = dict()
    models['lr'] = LogisticRegression()
    models['cart'] = DecisionTreeClassifier()
    models['knn'] = KNeighborsClassifier()
    models['rf'] = RandomForestClassifier(n_estimators=1000, # Ilość słabych estymatorów
                                      max_depth=2, # Maksymalna wysokość drzewa w słabym estymatorze
                                      min_samples_split = 2, # Minimalna ilość obserwacji wymagana do podziału węzła
                                      max_features = 3, # Maksymalna ilość zmiennych brana pod uwagę przy podziale węzła
                                      random_state=0,
                                      n_jobs = -1)
    models['aboost'] = AdaBoostClassifier(random_state=1)
    models['gboost'] = GradientBoostingClassifier(random_state=1,
                                      learning_rate=0.01)
    models['xgb'] = XGBClassifier(random_state=1,
                        learning_rate=0.01, # Szybkość "uczenia" się
                        booster='gbtree', # Jaki model wykorzystujemy (drzewo - gbtree, liniowe - gblinear)
                        max_depth=4 # Maksymalna głębokość drzewa 
                        )
    models['svm'] = SVC()
    models['bayes'] = GaussianNB()
    models['stacking'] = get_stacking()

    return models

In [ ]:
# ocena modeli przy wykorzystaniu cross-validation
def evaluate_model(model, X, y):
    
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')

    return scores

In [ ]:
# modele do evaluacji
models = get_models()
# ocena modeli
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X, y)
    results.append(scores)
    names.append(name)
    print('%8s mean-%.3f std-%.3f' % (name, np.mean(scores), np.std(scores)))

      lr mean-0.722 std-0.037
    cart mean-0.611 std-0.064
     knn mean-0.618 std-0.046
      rf mean-0.713 std-0.045
  aboost mean-0.675 std-0.045
  gboost mean-0.723 std-0.050


In [ ]:
# plot acc dla modeli i stacking
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

### Wszystkie dzisiejsze metody można znaleźć dokładnie (i przystępnie!) omówione na StatQuest: https://www.youtube.com/user/joshstarmer